## 003. IMDB
- 영화 리뷰 긍/부정 판단 위한 sentiment analysis dataset
- 25000 train data, 25000 test data로 구성
- torxhtext를 이용해 데이터 다운로드하기

``` Python
! pip install torchtext=0.15.2 # 자연어 처리 작업
! pip install portalocker=2.7.0 # 여러 프로세스가 동일한  파일을 동시에 접근할 때 충돌 방지
! pip install accelerate -U # pytorch 모델의 학습 및 평가 가속화
```

In [4]:
!pip install torchtext==0.15.2

In [5]:
!pip install portalocker==2.7.0

  Using cached portalocker-2.7.0-py2.py3-none-any.whl.metadata (6.8 kB)
Using cached portalocker-2.7.0-py2.py3-none-any.whl (15 kB)


In [6]:
!pip install accelerate -U

  Using cached accelerate-0.27.2-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)


In [1]:
# 위까지 실행 후 restart
from torchtext.datasets import IMDB

train_iter = IMDB(split='train')
test_iter = IMDB(split='test')



In [2]:
import random

random.seed(6)

# xxx_iter를 리스트로 변경
train_list = list(train_iter)
test_list = list(test_iter)

# 리스트 중 1000개씩 랜덤샘플링
train_list_small = random.sample(train_list, 1000)
test_list_small = random.sample(test_list, 1000)

# 각 변수의 첫번째 원소 출력
print(train_list_small[0])
print(test_list_small[0])

(2, "I LOVED this movie! I am biased seeing as I am a huge Disney fan, but I really enjoyed myself. The action takes off running in the beginning of the film and just keeps going! This is a bit of a departure for Disney, they don't spend quite as much time on character development (my husband pointed this out)and there are no musical numbers. It is strictly action adventure. I thoroughly enjoyed it and recommend it to anyone who loves Disney, be they young or old.")
(1, 'This was an abysmal show. In short it was about this kid called Doug who guilt-tripped a lot. Seriously he could feel guilty over killing a fly then feeling guilty over feeling guilty for killing the fly and so forth. The animation was grating and unpleasant and the jokes cheap. <br /><br />It aired here in Sweden as a part of the "Disney time" show and i remember liking it some what but then i turned 13.<br /><br />I never got why some of the characters were green and purple too. What was up with that? <br /><br />Tru

## 005. label encoding
위의 데이터셋은 (텍스트, 레이블) 형태로 구성되어 있으며, 레이블의 경우 2는 긍정, 1은 부정으로 구성되어 있다. 긍정을 1, 부정을 0으로 바꾸고 이걸 각각 train_texts, train_labels, test_texts, test_labels에 저장하라

In [3]:
train_texts  = []
train_labels = []
test_texts = []
test_labels = []

In [4]:
for label, text in train_list_small:
    train_labels.append(1 if label==2 else 0)
    train_texts.append(text)

In [5]:
for label, text in test_list_small:
    test_labels.append(1 if label==2 else 0)
    test_texts.append(text)

In [6]:
print(train_labels[0])
print(train_texts[0])

1
I LOVED this movie! I am biased seeing as I am a huge Disney fan, but I really enjoyed myself. The action takes off running in the beginning of the film and just keeps going! This is a bit of a departure for Disney, they don't spend quite as much time on character development (my husband pointed this out)and there are no musical numbers. It is strictly action adventure. I thoroughly enjoyed it and recommend it to anyone who loves Disney, be they young or old.


## 006. 학습 및 검증 데이터 분리
1000개를 8:2로 나누어라

In [7]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=3)

print(len(train_labels))
print(len(train_texts))

800
800


## 007. 토크나이징 및 인코딩
위에서 추출한 train, val, test 데이터를 pretrained distilbert-base-uncased 모델에 투입하기 위해 토크나이저를 사용해서 인코딩하라

In [9]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

tokenizer_config.json: 100%|██████████| 28.0/28.0 [00:00<00:00, 28.0kB/s]
c:\Users\User\Desktop\스터디\Kaggle_Transcription_Study\.conda\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
vocab.txt

In [10]:
# 토크나이저 실행
train_encodings = tokenizer(train_texts, truncation=True, padding=True) # truncation: 모델의 default max_length를 넘는 입력 부분을 더 이상 받지 않고 절단함
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

print(train_encodings['input_ids'][0][:5])

[101, 4937, 11350, 2038, 2048]


In [16]:
print(train_encodings.keys())

dict_keys(['input_ids', 'attention_mask'])


In [11]:
print(train_encodings['input_ids'][0][:5])
print(tokenizer.decode(train_encodings['input_ids'][0][:5]))

[101, 4937, 11350, 2038, 2048]

[CLS] cat soup has two


## 008. 데이터세트 클래스 생성
- torch.utils.data.Dataset을 상속하는 IMDBDataset이라는 클래스 작성하기
- 문제 007의 imdb 데이터셋에서 학습한 train/val/test encodings를 입력해서 클래스 인스턴스화하기

cf. __init__은 인스턴스화에 생기는 generator이다. generator는 클래스로 객체를 생성할 때 자동으로 호출되는 메서드로, 객체를 미리 설정한 값으로 초기화한다. 그리고 클래스를 구성하는 arguments, variable, function을 정의한다.


In [ ]:
import torch

class IMDBDataset(torch.utils.data.dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return 